In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Miten kommentti koostuu foorumilla?

In [ ]:
# Nesteen keskustelun linkki
r = requests.get("https://keskustelut.inderes.fi/t/harvia-foorumi-eli-haarumi-kansainvalista-kasvua-ja-hyvinvoinnin-megatrendeja/471/4")

# Jos status code = 200, vastaus saatiin
print(r)

# HTML:n jäsentäminen
soup = BeautifulSoup(r.content, 'html.parser')

# Haetaan tarvittavat elementit eli kommentit, aikaleima ja käyttäjä
comment = soup.find("div", {"class": "post"}).get_text(strip=True)
time = soup.find("time", {"class":"post-time"})
datetime = time['datetime']
creator = soup.find("span", {"class":"creator"})
name = creator.find("span", {"itemprop":"name"}).get_text(strip=True)

print(comment)
print(datetime)
print(name)

<Response [200]>
Hyvä Roasti tänään, vaikka olikin aika kevyt eikä hikeä tullu toimarilta. Harvialla todella puhtaat paperit ja tylsyys on hyvä sijoittamisessa, kuten Sammon kohdalla… Jatkoon… aloitan ostoprojektin huomenna.
2018-09-20T21:17:41Z
Kalle_P


## Kaikkien ketjun kommenttien hakeminen

In [76]:
# Listat kommenteille ja aikaleimoille

comments = []
dates = []
users = []

# Haetaan kaikki kommentit, joita on x kappaletta

for i in range(1,7997):
    r = requests.get(f"https://keskustelut.inderes.fi/t/harvia-foorumi-eli-haarumi-kansainvalista-kasvua-ja-hyvinvoinnin-megatrendeja/471/{i}")
    soup = BeautifulSoup(r.content, 'html.parser')

    # Kommentit
    comment = soup.find("div", {"class": "post"}).get_text(strip=True)
    comments.append(comment)

    # Aikaleimat
    time = soup.find("time", {"class":"post-time"})
    datetime = time['datetime']
    dates.append(datetime)
    
    # Käyttäjät
    creator = soup.find("span", {"class":"creator"})
    name = creator.find("span", {"itemprop":"name"}).get_text(strip=True)
    users.append(name)

In [101]:
# Siirretään kommentit dataframeihin

df_comments = pd.DataFrame({
    "datetime":dates,
    "user":users,
    "comment":comments
})

In [102]:
df_comments

,datetime,user,comment
0,2018-09-20T12:19:18Z,Verneri_Pulkkinen,Harvian toimari oli tänäänROASTissaja huomasin...
1,2018-09-20T12:43:56Z,timontti,"Harkinnassa, roast herätti mielenkiinnon ja la..."
2,2018-09-20T16:03:04Z,Markus14,"Karsastan tuota liikearvopommia, mikä yhtiöllä..."
3,2018-09-20T21:17:41Z,Kalle_P,"Hyvä Roasti tänään, vaikka olikin aika kevyt e..."
4,2018-09-21T05:25:08Z,Mikael_Rautanen,Vaikuttaa kyllä kaikin puolin laadukkaalta ja ...
...,...,...,...
7991,2025-02-24T16:19:50Z,kimsku,"Kiva kun pidetään ketjua aktiivisena, mutta re..."
7992,2025-02-24T16:44:17Z,Real,"onhan siinä HS:n kuvassa kiviä keskellä,HS-jut..."
7993,2025-02-26T17:44:22Z,Saapasjalkakissa,Mielenkiintoista tietoa pienen kannustimen vai...
7994,2025-02-27T05:14:06Z,eesau,Tässä vielä linkki lähteenä olevaan Ympäristöm...


In [ ]:
# Onko duplikaatteja?

df_comments.nunique()

datetime    7095
user         830
comment     7084
dtype: int64

In [ ]:
# Datassa on duplikaatteja, jotka näyttävät johtuvan siitä, että osa kommenteista on hävinnyt välistä
# Eli esim. kommenttia numero 10 ei ole enää
# Tämä mahdollisesti sekoittaa koodin ja luo duplikaatteja
# Riittävä taso minulle on vain poistaa duplikaatit

df_comments_unique = df_comments.drop_duplicates()

In [ ]:
# Nyt kommenttien määrä vastaa kommenttien määrää, joka foorumilla on datan keräys hetkellä näkyvissä

len(df_comments_unique)

7097

In [115]:
# Viedään kommentit talteen csv-tiedostoon

df_comments_unique.to_csv("comments.csv", index=False)

In [2]:
# Haetaan tiedostosta data

df = pd.read_csv("comments.csv")
df

,datetime,user,comment
0,2018-09-20T12:19:18Z,Verneri_Pulkkinen,Harvian toimari oli tänäänROASTissaja huomasin...
1,2018-09-20T12:43:56Z,timontti,"Harkinnassa, roast herätti mielenkiinnon ja la..."
2,2018-09-20T16:03:04Z,Markus14,"Karsastan tuota liikearvopommia, mikä yhtiöllä..."
3,2018-09-20T21:17:41Z,Kalle_P,"Hyvä Roasti tänään, vaikka olikin aika kevyt e..."
4,2018-09-21T05:25:08Z,Mikael_Rautanen,Vaikuttaa kyllä kaikin puolin laadukkaalta ja ...
...,...,...,...
7092,2025-02-24T16:19:50Z,kimsku,"Kiva kun pidetään ketjua aktiivisena, mutta re..."
7093,2025-02-24T16:44:17Z,Real,"onhan siinä HS:n kuvassa kiviä keskellä,HS-jut..."
7094,2025-02-26T17:44:22Z,Saapasjalkakissa,Mielenkiintoista tietoa pienen kannustimen vai...
7095,2025-02-27T05:14:06Z,eesau,Tässä vielä linkki lähteenä olevaan Ympäristöm...


In [14]:
# Haetaan kirjasto kääntämiseen

from deep_translator import GoogleTranslator

In [4]:
# Luodaan käännökselle testisetti

df_10 = df.head(10)
df_10

,datetime,user,comment
0,2018-09-20T12:19:18Z,Verneri_Pulkkinen,Harvian toimari oli tänäänROASTissaja huomasin...
1,2018-09-20T12:43:56Z,timontti,"Harkinnassa, roast herätti mielenkiinnon ja la..."
2,2018-09-20T16:03:04Z,Markus14,"Karsastan tuota liikearvopommia, mikä yhtiöllä..."
3,2018-09-20T21:17:41Z,Kalle_P,"Hyvä Roasti tänään, vaikka olikin aika kevyt e..."
4,2018-09-21T05:25:08Z,Mikael_Rautanen,Vaikuttaa kyllä kaikin puolin laadukkaalta ja ...
5,2018-09-21T09:34:04Z,anon38833097,Vähän tuli ostettuaDefensiivisyyttä myös salkk...
6,2018-10-11T12:55:29Z,HerrDax,Mikäs erityisesti Harviaa tänä mielenkiintoise...
7,2018-10-11T13:05:34Z,timontti,"Pakko yrittää pilkkiä alta 5,4€ tätä, vaikka j..."
8,2018-10-11T13:21:33Z,Koli,Eiköhän se syy ollut juurikin tuossa mainitsem...
9,2018-10-12T10:14:34Z,Markus14,"“Osinko maksetaan osakkeenomistajalle, joka on..."


In [34]:
# Poistetaan dataframesta rivit, joilla comment on NULL

df = df[df["comment"].isnull()== False].reset_index(drop=True)

In [35]:
# Luodaan dataframe, jossa on kommentit, joissa alle 5 000 merkkiä

df_under_5000_chars = df[df["comment"].str.len() < 5000].reset_index(drop=True)

In [40]:
# Luodaan dataframeen uusi sarake, jossa kommentti

translator = GoogleTranslator(source='fi', target='en')

df_under_5000_chars["comment_en"] = df_under_5000_chars["comment"].apply(lambda x: translator.translate(x))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/weakref.py:575: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  info = self._Info()


TranslationNotFound: Ahonen myi kyllä nyt syyskuun aikana 20 000kpl lappuja, alkuvuonna vielä meni jotain 1500 lisää. Eli lähemmäs miljoonalla hän on myynyt kyllä tänä vuonna, joka on jo aika iso potti.Ahosella taitaa olla takana jo parinkymmenen vuoden työhistoria Harviasta. Hän ei taida olla mikään sijoittaja luonteeltaan, joten nämä myynit liittyy todennäköisesti vain tarpeeseen rahastaa. --> No translation was found using the current translator. Try another translator?

In [38]:
len(df[df["comment"].str.len() >= 5000])

27

In [41]:
df_under_5000_chars

,datetime,user,comment
0,2018-09-20T12:19:18Z,Verneri_Pulkkinen,Harvian toimari oli tänäänROASTissaja huomasin...
1,2018-09-20T12:43:56Z,timontti,"Harkinnassa, roast herätti mielenkiinnon ja la..."
2,2018-09-20T16:03:04Z,Markus14,"Karsastan tuota liikearvopommia, mikä yhtiöllä..."
3,2018-09-20T21:17:41Z,Kalle_P,"Hyvä Roasti tänään, vaikka olikin aika kevyt e..."
4,2018-09-21T05:25:08Z,Mikael_Rautanen,Vaikuttaa kyllä kaikin puolin laadukkaalta ja ...
...,...,...,...
7054,2025-02-24T16:19:50Z,kimsku,"Kiva kun pidetään ketjua aktiivisena, mutta re..."
7055,2025-02-24T16:44:17Z,Real,"onhan siinä HS:n kuvassa kiviä keskellä,HS-jut..."
7056,2025-02-26T17:44:22Z,Saapasjalkakissa,Mielenkiintoista tietoa pienen kannustimen vai...
7057,2025-02-27T05:14:06Z,eesau,Tässä vielä linkki lähteenä olevaan Ympäristöm...


In [28]:
len(df)

7097

In [ ]:
# Rivit, joilla "comment" on NaN

len(df[df["comment"].isnull()==True])

11